In [ ]:
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import cmocean as cmo
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
#from matplotlib.patches import Polygon
#from copy import copy


In [ ]:
vmax = 100
vmin = -10
linthresh = .3
linscale = .25

fracpos = (np.log10(vmax/linthresh)+linscale)/(np.log10(vmax/linthresh)+np.log10(-(vmin/linthresh))+2*linscale)
nneg = np.int_((1-fracpos)*256)

colors1 = plt.get_cmap('cmo.dense_r')(np.linspace(0,1.,nneg+1))
colors2 = plt.get_cmap('gist_heat_r')(np.linspace(0., 1, 256-nneg-1))

# combine them and build a new colormap
colors = np.vstack((colors1, colors2))
mymap = mpl.colors.LinearSegmentedColormap.from_list('my_colormap', colors)

norm=mpl.colors.SymLogNorm(linthresh, vmin=vmin, vmax=vmax, linscale=linscale)

cmap2 = plt.get_cmap('cmo.ice')
norm2 = mpl.colors.Normalize(vmin=100,vmax=800)

cmap3 = plt.get_cmap('cmo.rain')
norm3 = mpl.colors.Normalize(vmin=0,vmax=1)

In [ ]:
runname = 'MISMIPplus_5km_runtime_FCMP'

In [ ]:
ds0 = xr.open_dataset(f'../output/{runname}/main_output_ANT_grid.nc')
ds0['x'] = ds0.x/1000
ds0['y'] = ds0.y/1000

In [ ]:
ymin = -800
ymax = 1000
ymax2 = 250

lwborder = 2.

j0 = np.argmin(ds0.y.data**2)
xsecs = [400,450,500]
i0 = []
for x in xsecs:
    i0.append(np.argmin((ds0.x.data-x)**2))

cols = ['tab:red','tab:blue','tab:orange']
col2 = 'tab:green'


In [ ]:
f = 0

fig = plt.figure(figsize=(10,5))

gs = fig.add_gridspec(len(xsecs),2,  width_ratios=(1, 1), height_ratios=(np.ones(3)),
                    left=0.1, right=0.9, bottom=0.1, top=0.9,
                    wspace=0.15, hspace=0.1)

for t in range(len(ds0.time)):
    #Make up figure
    
    # section along flow
    ax = fig.add_subplot(gs[:,0])
    ax.plot(ds0.x,ds0.Hib[0,j0,:],color='.9',zorder=9)
    ax.plot(ds0.x,ds0.Hs[0,j0,:],color='.9',zorder=9)

    dds = ds0.isel(time=t,y=j0)

    ax.fill_between(dds.x,ymin,dds.Hb,color='brown')
    ax.fill_between(dds.x,dds.Hb,dds.Hib,color='teal')
    ax.fill_between(dds.x,dds.Hib,dds.Hs,color='.9')
    ax.fill_between(dds.x,dds.Hs,ymax,color='lightblue')

    for i,x in enumerate(xsecs):
        ax.plot([x,x],[ymin,ymax2],c=cols[i],lw=lwborder,zorder=10)

    ax.set_xlim(300,650)
    ax.set_ylim(ymin,ymax)

    for sp in ['bottom','top','left','right']:
        ax.spines[sp].set_color(col2)
        ax.spines[sp].set_linewidth(lwborder)
        ax.spines[sp].set_zorder(10)

    # sections across flow
    for i,ii in enumerate(i0):
        ax = fig.add_subplot(gs[i,1])
        ax.plot(ds0.y,ds0.Hib[0,:,ii],color='.9')
        ax.plot(ds0.y,ds0.Hs[0,:,ii],color='.9')

        dds = ds0.isel(time=t,x=ii)
        ax.fill_between(dds.y,ymin,dds.Hb,color='brown')
        ax.fill_between(dds.y,dds.Hb,dds.Hib,color='teal')
        ax.fill_between(dds.y,dds.Hib,dds.Hs,color='.9')
        ax.fill_between(dds.y,dds.Hs,ymax,color='lightblue')

        ax.plot([0,0],[ymin,ymax2],c=col2,lw=lwborder,zorder=10)

        ax.set_ylim(ymin,ymax2)
        ax.set_xlim(-40,40)
        if ii<i0[-1]:
            ax.set_xticklabels([])
        ax.invert_xaxis()

        for sp in ['bottom','top','left','right']:
            ax.spines[sp].set_color(cols[i])
            ax.spines[sp].set_linewidth(lwborder)
            ax.spines[sp].set_zorder(10)

    fig.suptitle(f"Year {ds0.time[t].data-20000:10.2f}")

    canvas = FigureCanvas(fig)
    canvas.print_figure(f'../video/frame_{f:03d}.png',dpi=300)

    f += 1
    plt.clf()

In [ ]:
#Make video
framerate = 40

command = f'ffmpeg -r {framerate} -f image2 -i ../video/frame_%03d.png -pix_fmt yuv420p -vcodec libx264 -crf 24 ../video/sections_{runname}.mp4'
print(command)

os.system(command)
os.system('rm -r ../video/frame_*.png')